Intel Corporation Interactive Event Analysis
=========================================

This interactive notebook can be used for exploring events generated by Vertex.AI components.

To use it, you'll first need to produce an event log.  The way you do this varies with the various components (since they're configured in different ways).

* With the Tile server, you'll want to add something like the following stanza to your JSON _server.conf_:

      "eventlog": {
        "@type": "type.vertex.ai/vertexai.eventing.file.proto.EventLog",
        "filename": "eventlog.gz"
       },
       "event_tracking_mode": "EVENT_TRACKING_MODE_GLOBAL"
     
  When run with this configuration, the server will write a file, _eventlog.gz_, on shutdown (which you can trigger with a Ctrl-C).

     
* With the PlaidML Python backends, you can typically set an environment variable:

       PLAIDML_EVENTLOG_FILENAME=eventlog.gz
     
  This will cause events to be logged to _eventlog.gz_ on shutdown.

Next, run this notebook with:

    ./t2 run //tools/analysis
    
By default, the notebook will look for _eventlog.gz_ in your home directory.  If that's wrong, you can pass a filename like so:

    ./t2 run //tools/analysis -- /path/to/eventlog.gz
    
Once the notebook is up and running you can start adding your own cells to explore the output data.

In [ ]:
# Setup: Add some useful imports, &c.
import os
import platform
import sys

if platform.system() != 'Windows':
    ta_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
    if ta_dir not in sys.path:
        sys.path.insert(1, ta_dir)

import bokeh.io as bi
import bokeh.models as bm
import bokeh.plotting as bp
import bokeh.palettes as bpa
import bokeh.transform as bt
from collections import defaultdict, namedtuple
import datetime
import functools
import graphviz
import humanize
import imp
import IPython.display
from IPython.display import display_svg
import ipywidgets as widgets
from ipywidgets import interact
import itertools
import networkx as nx
import numpy as np
import pandas as pd
from pygments import highlight
from pygments.lexers.c_cpp import CLexer
from pygments.formatters import HtmlFormatter

bi.output_notebook()

In [ ]:
# With the Python path configured, the Bazel-built code can be loaded.
import tools.analysis as ta

In [ ]:
# Load the raw events, and make them useful.
scope = ta.Scope()
scope.read_eventlog(os.getenv('PLAIDML_EVENTLOG_FILENAME'))
acts = ta.as_dataframe(scope, scope.activities)

In [ ]:
# Let's build up some stats on the kernels:
execs = acts[acts.verb.isin(ta.VERBS_EXECUTING)]
halinfos = execs.activity.apply(lambda d: scope.get_activity(d, d.ocl_runinfo.kernel_id).ocl_kernelinfo)
groups = pd.DataFrame({
    'halinfo': halinfos,
    'runtime': execs.end - execs.start,
    'kname': halinfos.apply(lambda i: i.kname),
    'activity': execs.activity.apply(lambda d: scope.get_activity(d, d.ocl_runinfo.kernel_id)),
}).groupby('kname')
kernels = groups.mean()
halinfos = groups.first().loc[:, 'halinfo']
kernels.sort_values(by='runtime', ascending=False, inplace=True)
total = np.sum(kernels.runtime)

kernels = kernels.assign(
    halinfo=halinfos,
    activity=groups.first().loc[:, 'activity'],
    compilation=groups.first().loc[:, 'activity'].apply(lambda a: a.hal_compilationinfo),
    build=groups.first().loc[:, 'activity'].apply(lambda a: a.ocl_buildinfo),
    c_runtime=kernels.runtime.cumsum(),
    percentage=kernels.runtime / total,
    gflops=halinfos.apply(lambda x: x.kinfo.flops) / kernels.runtime,
    io=halinfos.apply(lambda x: x.kinfo.bytes) / kernels.runtime,
    vec_sizes=halinfos.apply(lambda x: x.kinfo.contraction.vec),
    offsets=halinfos.apply(lambda x: x.kinfo.contraction.off),
    accesses=halinfos.apply(lambda x: [(a.name, a.range, '.'.join(str(s) for s in a.strides)) for a in x.kinfo.contraction.accesses]),
    constraints=halinfos.apply(lambda x: ['{} <= {}'.format('+'.join(['{}*{}'.format(v, x.kinfo.contraction.accesses[i].name) for i, v in enumerate(c.lhs) if v != 0]),
                                                            c.rhs) for c in x.kinfo.contraction.constraints])
)

kernels = kernels.assign(c_percentage=kernels.percentage.cumsum())

kernels_fmt = {
    'gflops': lambda x: humanize.naturalsize(x, gnu=True, format='%.1f ') + 'F/s',
    'io': lambda x: humanize.naturalsize(x) + '/s',
    'percentage': lambda x: '{:.2f}%'.format(100*x),
    'c_percentage': lambda x: '{:.2f}%'.format(100*x),
    'runtime': lambda x: datetime.timedelta(seconds=x),
    'c_runtime': lambda x: datetime.timedelta(seconds=x),
    'accesses': lambda x: ' --- '.join(['{} : {}[{}]'.format(*a) for a in x]),
    'constraints': lambda x: ', '.join(x)
}

# Browser-side copy of the kernels DataFrame.  N.B. This must only contain serializable data.
kernels_source = bm.ColumnDataSource(kernels[['c_runtime', 'c_percentage', 'gflops', 'io', 'percentage', 'runtime']])

# The default tooltip to use when displaying kernels graphically.
kernels_hover = bm.HoverTool(tooltips=[
    ('kernel', '@kname'),
    ('knum', '$index'),
    ('gflops', '@gflops'),
    ('GB/s', '@io'),
])

In [ ]:
# Let's display mean OpenCL kernel execution times, because they're often interesting.
kernels[['runtime', 'c_runtime', 'percentage', 'c_percentage', 'gflops', 'io']].style.format(kernels_fmt)


In [ ]:
# Here's a graph of the kernels.  Note the knum is ordered by highest runtime.
p = bp.figure(title = 'Kernel Performance', plot_width=760, plot_height=800)
p.add_tools(kernels_hover)
p.xaxis.axis_label = 'GFlops'
p.yaxis.axis_label = 'GB/s'

circle_size_transform = bm.transforms.CustomJSTransform(
    func='return (x) * 2000',
    v_func = '''
      new_xs = new Array(xs.length)
      for(i = 0; i < xs.length; i++) {
        new_xs[i] = xs[i] * 2000
      }
      return new_xs''')

p.circle(x='gflops', y='io', fill_alpha=0.2, size=bt.transform('percentage', circle_size_transform),
         source=kernels_source)

bp.show(p)

In [ ]:
# We can also view the mean kernel execution times as a bar graph.
p = bp.figure(title = 'Kernel Mean Runtimes',
              plot_width=760, plot_height=kernels.shape[0]*12,
              x_range=(0., kernels.iloc[0].runtime * (1.15 + kernels.iloc[0].c_percentage)),
              y_range=kernels.iloc[::-1].index.values,
              x_axis_location='above')
p.extra_x_ranges = {'Percent': bm.ranges.DataRange1d(flipped=True)}
p.add_tools(kernels_hover)
p.xaxis.axis_label = 'Mean Runtime (s)'
p.xaxis.formatter = bm.formatters.NumeralTickFormatter(format='0.000000')
p.yaxis.axis_label = 'Kernel'
p.add_layout(bm.axes.LinearAxis(x_range_name='Percent', axis_label='Cumulative Percentage',
                                formatter=bm.formatters.NumeralTickFormatter(format='0%')), 'above')

p.hbar(y='kname', height=1, left=0, right='runtime', fill_alpha=0.2, source=kernels_source)

p.hbar(y='kname', height=1, left=0, right='c_percentage', fill_alpha=0.2,
       fill_color='green', line_color='green', x_range_name='Percent', source=kernels_source)

bp.show(p)

In [ ]:
# Here's a kernel source viewer.  We order the kernels greatest-runtime-first, since that's usually
# what's most interesting.

def KernelWidget(krow):
    (halinfo, runtime, accesses, constraints) = krow[['halinfo', 'runtime', 'accesses', 'constraints']]
    code = """
// Kernel: {}
// Mean Runtime: {}
// Operations:{}
// Accesses:{}
// Constraints:{}
//
{}
""".format(halinfo.kname, datetime.timedelta(seconds=runtime),
           ''.join(['\n//   {}'.format(op) for op in halinfo.kinfo.contraction.ops]),
           ''.join(['\n//   {} range={} strides=[{}]'.format(*a) for a in accesses]),
           ''.join(['\n//   {}'.format(c) for c in constraints]),
           halinfo.src)
    html = highlight(code, CLexer(), HtmlFormatter(full=True))
    return widgets.HTML(html, layout=widgets.Layout(width='100%', height='500px'))

@interact(knum=widgets.IntSlider(min=0, max=kernels.shape[0]-1, continuous_update=False))
def src(knum=0):
    return KernelWidget(kernels.iloc[knum])

In [ ]:
# Here's the program of the highest-runtime kernel:
kernel = kernels.iloc[0]
kname = kernel.halinfo.kname
total = 0
for size, count in kernel.compilation.alloc_sizes.items():
    total += size*count
print('Kernel {}\'s program uses {} of temporary memory.\n'.format(kname, humanize.naturalsize(total, binary=True)))
print(kernel['compilation'].program.code)

# N.B. If you want memory details, you can do something like this:
# for size, count in kernel.compilation.alloc_sizes.items():
#     print('  {} alloc(s) of size {}'.format(count, size))

In [ ]:
# Here's the device for that longest-running kernel:
kact = kernels.iloc[0].activity
scope.get_activity(kact, kact.ocl_buildinfo.device_id).ocl_deviceinfo

In [ ]:
# And the plaform for that device:
kact = kernels.iloc[0].activity
dact = scope.get_activity(kact, kact.ocl_buildinfo.device_id)
scope.get_activity(dact, dact.ocl_deviceinfo.platform_id).ocl_platforminfo

In [ ]:
# Here's a network view of the static computation schedule for the longest-running kernel's program.
#
# Unfortunately, the static computation schedule for even a simple network can be rather large.
# So: this implementation constructs subgraphs of a particular radius around nodes of interest,
# and visualizes the subgraphs.

# Build the overall computation graph, using networkx.
kact = kernels.iloc[0].activity
spb = kact.lpt_schedule
sched = nx.DiGraph()

nidx_iter = itertools.count(1)
aidx_nidx = {}
sidx_nidx = {}
first_use = {}

def add_alloc(aidx, alloc=None):
    if not alloc:
        alloc = spb.allocs[aidx]
    label='a{}'.format(aidx)
    if alloc.input:
        label = label + ' input:' + alloc.input
    if alloc.output:
        label = label + ' output:' + alloc.output
    nidx = next(nidx_iter)
    aidx_nidx[aidx] = nidx
    sched.add_node(nidx, label=label, shape='box')
    return nidx

for aidx, alloc in enumerate(spb.allocs):
    add_alloc(aidx, alloc)

for sidx, step in enumerate(spb.steps):
    nidx = next(nidx_iter)
    sidx_nidx[sidx] = nidx
    if step.HasField('run'):
        kname = spb.knames[step.run.kidx]
        if not kname in first_use:
            first_use[kname] = sidx
    sched.add_node(nidx, shape='ellipse', label='s{}'.format(sidx))
    for aidx in step.run.input_aidxs:
        sched.add_edge(aidx_nidx[aidx], nidx)
    for aidx in step.run.output_aidxs:
        sched.add_edge(nidx, add_alloc(aidx))

def visualize_subgraph(radius=3, sidx=0):
    # Extract the desired subgraph, and render it with graphviz.
    subg = nx.ego_graph(sched, sidx_nidx[sidx], radius=radius+1, undirected=True)
    vizg = nx.ego_graph(sched, sidx_nidx[sidx], radius=radius, undirected=True)
    viz = graphviz.Digraph(graph_attr={'dpi': '56'})
    incoming = set()
    outgoing = set()
    for u, v in subg.edges:
        viz.edge('n{}'.format(u), 'n{}'.format(v))
        incoming.add(u)
        outgoing.add(v)
    internal = incoming.intersection(outgoing)
    for nidx, ndata in subg.nodes.items():
        if nidx not in vizg.nodes and nidx not in internal:
            shape = 'point'
        else:
            shape = ndata['shape']
        viz.node('n{}'.format(nidx), ndata['label'], shape=shape)
    out = widgets.Output()
    with out:
        try:
            display_svg(viz._repr_svg_(), raw=True)        
        except graphviz.ExecutableNotFound as not_found:
            return not_found
    w = [out]
    if spb.steps[sidx].HasField('run'):
        kname = spb.knames[spb.steps[sidx].run.kidx]
        w.append(widgets.Label('Step {0} => {1}, first used by step {2}'.format(sidx, kname, first_use[kname])))
        w.append(KernelWidget(kernels.loc[kname]))
    return widgets.VBox(w)

interact(visualize_subgraph,
         radius=widgets.IntSlider(value=2, min=1, max=20, continuous_update=False),
         sidx=widgets.BoundedIntText(value=0, min=0, max=len(spb.steps)-1,
                                     continuous_update=False,
                                     description='Step[0..{}] '.format(len(spb.steps)-1)));

In [ ]:
# Let's take a look at the invocations.  It can be useful to compare the sum
# kernel execution time to the end-to-end kernel execution time:
invocations = acts[acts.verb == 'plaidml::invoker::ScheduleInvocation']

def kernel_stats(row):
    inv = row.activity
    execs = ta.as_dataframe(scope, filter(lambda act: act.verb in ta.VERBS_EXECUTING, inv.subtree()))
    execs = execs.assign(delta=execs.end-execs.start)
    aggs = execs.agg({'start': np.min, 'end': np.max, 'delta': np.sum})
    overall_time = aggs.end - aggs.start
    row = row.drop('parent')
    return row.append(pd.Series([execs.size, overall_time, aggs.delta, aggs.delta/overall_time],
                                index=['kernels', 'kernel_walltime', 'kernel_exectime', 'load']))

invocations = invocations.apply(kernel_stats, axis='columns').sort_values(by=['kernels', 'load'], ascending=False)
invocations[['verb', 'kernels', 'start', 'end', 'kernel_walltime', 'kernel_exectime', 'load']].style.format({
    'kernel_walltime': lambda x: datetime.timedelta(seconds=x),
    'kernel_exectime': lambda x: datetime.timedelta(seconds=x),
    'start': lambda x: datetime.timedelta(seconds=x),
    'end': lambda x: datetime.timedelta(seconds=x),
    'load': lambda x: '{:.2f}%'.format(100*x),
})

In [ ]:
# It can also be interesting to look at what happens across an entire batch invocation.

# Let's pick the last of the invocations with the most kernels -- that's usually what we're interested in:
inv = invocations[invocations.kernels == invocations.iloc[0].kernels].sort_values(by='start', ascending=False).iloc[0]

# Add in all current-buffer mapping events that start after the start of this invocation and before the next.
# (Not that there will typically be a next invocation.)
next_invs = invocations[inv.start < invocations.start].sort_values(by='start')
current_mask = (acts.verb == 'tile::MapCurrent') & (inv.start < acts.start)
if next_invs.shape[0]:
    current_mask = current_mask & (acts.start < next_invs.iloc[0].start)

# Add in all discard-buffer mapping events that start before the start of this invocation and after the previous.
prev_invs = invocations[invocations.start < inv.start].sort_values(by='start')

discard_mask = (acts.verb == 'vertexai::DiscardCurrent') & (acts.start < inv.start)
if prev_invs.shape[0]:
    discard_mask = discard_mask & (prev_invs.iloc[-1].start < acts.start)
    
mask = current_mask | discard_mask

# Build up the run.
run = ta.as_dataframe(scope, itertools.chain.from_iterable([a.subtree() for a in ([inv.activity] + list(acts[mask].activity.values))]))
run = run.assign(name=run.index.astype(str)+'/'+run.verb.apply(lambda s: s.rsplit('::', 1)[-1]))
run.update(run[['start', 'end']] - run.start.min())
             
# Rewrite some acts to have their parent's name, reducing the complexity of the output graph.
reparent = run[run.verb.isin(['tile::hal::opencl::HostQueue', 'tile::hal::opencl::DevQueue', 'tile::hal::opencl::Executing', 'tile::hal::opencl::Buffer::Unmap'])]
run.update(pd.merge(run, reparent, left_index=True, right_on='parent').name_x.rename('name'))

knames = pd.concat([acts.parent, run[run.verb.isin(ta.VERBS_EXECUTING)].activity.apply(
    lambda d: scope.get_activity(d, d.ocl_runinfo.kernel_id).ocl_kernelinfo.kname).rename('kname')], axis=1)
knames = knames.dropna().set_index(keys='parent')
run = pd.merge(run, knames, left_on='parent', right_index=True, how='outer')
run = run.assign(kname=run.kname.combine_first(knames.kname))
run = run.sort_values(by='start')
run_source = bm.ColumnDataSource(run[['verb', 'start', 'end', 'name', 'kname']])

# And plot them.
verb_cmap = bt.factor_cmap('verb', palette=bpa.Category20[20], factors=sorted(run.verb.unique()))
y_range=run.name.drop_duplicates()[::-1].values
p = bp.figure(y_range=y_range,
              plot_height=800,
              plot_width=800,
              title='Invocation Gantt',
              x_axis_location='above')
p.add_tools(bm.HoverTool(
    tooltips=[
        ('action', '@verb'),
        ('kname', '@kname'),
        ('start', '@start{0.000000}'),
        ('end', '@end{0.000000}')
    ]))
p.hbar(y='name', left='start', right='end', height=1, source=run_source, fill_alpha=0.2, fill_color=verb_cmap)

p.ygrid.grid_line_color = None
p.xaxis.axis_label = 'Time (seconds)'
p.xaxis.formatter = bm.formatters.NumeralTickFormatter(format='0.000000')
p.outline_line_color = None

bp.show(p)